In [39]:
import IPython, ipywidgets, jinja2, pandas
from coffeetools import coffee
get_ipython = ipywidgets.get_ipython

In [73]:
%%file _layouts/d3_helper.coffee
window.update = ( selection, data, tag = 'div',  classes = [] )->
    selection = selection.selectAll [tag, classes...].join '.'
        .data data
    selection.enter().append tag
    for c in classes
        selection.classed c, true
    selection.exit().remove()
    selection

Writing _layouts/d3_helper.coffee


In [84]:
%%file _layouts/table.html
<table id="catalog" class="table"></table>

Overwriting _layouts/table.html


In [85]:
%%file _layouts/update_table.coffee
debugger
data = {{df}}

table = d3.select '#catalog'
header = table.selectAll 'tr.header'
    .data [ '', data['columns'] ]
header.enter().append('tr').classed 'header', true
header.each (d)->
    update d3.select(@), d, 'th'
        .text (v)-> v
header.exit().remove()
value = update table, data['data'], 'tr', ['value']
value.each (row)->
    update d3.select(@), row, 'td'
        .text (v)-> JSON.stringify v

Overwriting _layouts/update_table.coffee


In [130]:
class Views( ):
    def __init__(self):
        self.env = jinja2.Environment( loader = jinja2.FileSystemLoader('_layouts') )
        self.template = {}
        for key in self.env.list_templates():
            self.template[key] = self.env.get_template( key )
            
    def process( self, template, df=pandas.DataFrame(), bare=False, display=True, *args, **kwargs):
        processing = self.template[template].render( df=df.to_json(orient='split'), *args, **kwargs )
        if template.endswith('.coffee'):
            processing = coffee.compile( processing, bare=bare )
        if display:
            if template.endswith('html'):
                return IPython.display.HTML( processing )
            if template.endswith('coffee') or template.endswith('js'):
                return IPython.display.Javascript( processing )

        return processing
    def layout( self, layout = [[]]):
        return ipywidgets.VBox(children=[ipywidgets.HBox( children=vbox ) for vbox in layout])
            
    def display( self, *args ):
        return IPython.display.display( *args )

In [131]:
v = Views()

In [132]:
butts = v.layout(
    [[ipywidgets.Button(description='A'), ipywidgets.Button(description='B')],
    [ipywidgets.Button(description='Aasdf'), ipywidgets.Button(description='Basdf'),ipywidgets.Button(description='Aasdf2asdf')]]
)

In [134]:
num_rows = ipywidgets.IntText( description='Number of Rows', value=10 )
row = ipywidgets.IntSlider( description='Current Row', value=0, min=0, max=iris.flowers.shape[0] )
def subsample( i ):
    IPython.display.display(v.process(
        v.template['update_table.coffee'].name, 
        df=iris.flowers.iloc[i['new']:i['new']+num_rows.value]
    ))
row.observe( names='value', handler= subsample )
v.display( butts, v.process(v.template['table.html'].name) , ipywidgets.Box( children=[num_rows, row]))

<IPython.core.display.Javascript object>

In [116]:
v.process(v.template['table.html'].name) 

In [ ]:
v.process(v.template['update_table.coffee'].name, df=iris.flowers.iloc[i:i+num_rows.value]) 

In [77]:
IPython.display.Javascript( v.process(v.template['d3_helper.coffee'].name) )

<IPython.core.display.Javascript object>

In [48]:
from bokeh.sampledata import iris

In [71]:
print(coffee.compile(v.template['d3_helper.html'].render()))

// Generated by CoffeeScript 1.10.0
(function() {
  var slice = [].slice;

  window.update = function(selection, data, tag, classes) {
    var c, i, len;
    if (tag == null) {
      tag = 'div';
    }
    if (classes == null) {
      classes = [];
    }
    selection = selection.selectAll([tag].concat(slice.call(classes)).join('.')).data(data);
    selection.enter().append(tag);
    for (i = 0, len = classes.length; i < len; i++) {
      c = classes[i];
      selection.classed(c, true);
    }
    selection.exit().remove();
    return selection;
  };

}).call(this);



In [72]:
v.process(v.template['d3_helper.html'].name)

"window.update = ( selection, data, tag = 'div',  classes = [] )->\n    selection = selection.selectAll [tag, classes...].join '.'\n        .data data\n    selection.enter().append tag\n    for c in classes\n        selection.classed c, true\n    selection.exit().remove()\n    selection"